In [1]:
!pip install -q transformers datasets librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [14]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn import metrics
from sklearn.metrics import classification_report

In [3]:
# !unzip 'disgust-20241215T174917Z-001.zip'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = load_dataset("csv", data_files={"train": "/content/drive/MyDrive/Проект/train.csv", "validation": "/content/drive/MyDrive/Проект/valid.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
valid_data = pd.DataFrame(data['validation'])

In [7]:
# для теста пока первые 5 строк
#valid_data = valid_data[:5]
valid_data.head()

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Это в Новом Орлеане?? Я действительно чувствую...,0,0,0,0,0,0,1
1,"Ты знаешь ответ, чувак, ты запрограммирован пе...",0,0,0,1,0,0,1
2,Мне никогда в жизни не было так грустно!,0,0,0,0,1,0,0
3,Экономика находится под жестким контролем и су...,0,0,0,1,0,0,1
4,Он мог бы легко взять настоящую камеру из зако...,0,0,0,1,0,0,0


In [8]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
#emotion_labels = ["anger" , "disgust"]

In [9]:
# Функция для предсказания эмоций
def predict_emotions(texts):
    emotion_labels = ['not' + emotion, emotion]
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

    # Преобразуем вероятности в словарь
    return [{emotion: prob.item() for emotion, prob in zip(emotion_labels, probabilities[i])} for i in range(probabilities.size(0))]

In [10]:
# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
# создаем нулевой датасет для заполнения
df_valid = valid_data[['text']].copy()

for emotion in emotion_labels:
  df_valid[emotion] = 0

df_valid

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Это в Новом Орлеане?? Я действительно чувствую...,0,0,0,0,0,0,0
1,"Ты знаешь ответ, чувак, ты запрограммирован пе...",0,0,0,0,0,0,0
2,Мне никогда в жизни не было так грустно!,0,0,0,0,0,0,0
3,Экономика находится под жестким контролем и су...,0,0,0,0,0,0,0
4,Он мог бы легко взять настоящую камеру из зако...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
5421,"Это довольно опасно, когда государство решает,...",0,0,0,0,0,0,0
5422,"Сегодня утром я подала на развод. Надеюсь, он ...",0,0,0,0,0,0,0
5423,"В последний раз, когда это случилось, я просто...",0,0,0,0,0,0,0
5424,"Терпеть не могу этого высокомерного придурка, ...",0,0,0,0,0,0,0


In [25]:
for emotion in emotion_labels:
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Проект/' + emotion + '/')

  # Получение предсказаний на тестовой выборке
  predictions = predict_emotions(valid_data.text.to_list())

  df_tmp=pd.DataFrame(predictions)
  df_tmp.rename(columns={emotion: emotion + '_proba'}, inplace=True)

  df_tmp[emotion] = df_tmp[ emotion + '_proba'].apply(lambda x: 1 if x >= 0.3 else 0)

  result = pd.concat([valid_data['text'], df_tmp[[emotion + '_proba', emotion]]], axis=1)


  # Вывод предсказаний
  #for text, pred in zip(valid_data.text.to_list(), predictions):
  #  print(f"Text: {text}, Predictions: {pred}")
  df_valid[emotion]=result[emotion]
  df_valid



In [22]:
df_valid.to_csv("/content/drive/MyDrive/Проект/validation.csv", index=False)

In [26]:
true_labels = valid_data[["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]].values
predicted_labels = df_valid[["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]].values

report = classification_report(true_labels, predicted_labels, target_names=["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"])
print(report)

              precision    recall  f1-score   support

       anger       0.53      0.36      0.43       717
     disgust       0.36      0.27      0.31        97
        fear       0.64      0.44      0.52       105
         joy       0.73      0.82      0.77      2219
     sadness       0.58      0.51      0.54       390
    surprise       0.51      0.39      0.44       624
     neutral       0.57      0.66      0.61      1766

   micro avg       0.63      0.63      0.63      5918
   macro avg       0.56      0.49      0.52      5918
weighted avg       0.62      0.63      0.62      5918
 samples avg       0.60      0.65      0.61      5918



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
